In [1]:
import json
import os
import random
import cv2 as cv2
import numpy as np
import scipy.io
print(cv2.__version__)
from os.path import isfile, join
import matplotlib.pyplot as plt
import math

backSub = cv2.createBackgroundSubtractorMOG2(detectShadows=False, history=15,varThreshold=50) 

def getFrame(sec,vidcap):
    frame=[]
    bk=[]
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        frame = backSub.apply(image)
        bk = cv2.bitwise_or(image, image, mask=frame)

    return hasFrames,frame,image,bk

def scale_contour(cnt, scale):
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    cnt_norm = cnt - [cx, cy]
    cnt_scaled = cnt_norm * scale
    cnt_scaled = cnt_scaled + [cx, cy]
    cnt_scaled = cnt_scaled.astype(np.int32)

    return cnt_scaled

bkimg=[]
col_images = []
orginal_img=[]
face_cascade = cv2.CascadeClassifier('cars.xml')

vidcap = cv2.VideoCapture('test_video.mp4')
sec = 0
frameRate = 0.1
count=1
success,np_frame,image,bkim = getFrame(sec, vidcap)
while success:
    col_images.append(np_frame)
    orginal_img.append(image)
    bkimg.append(bkim)
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success,np_frame,image,bkim= getFrame(sec,vidcap)
    

for i in range(len(orginal_img)-1):
    grayA = cv2.cvtColor(bkimg[i], cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(bkimg[i+1], cv2.COLOR_BGR2GRAY)
    diff_image = cv2.absdiff(grayB, grayA)
    blur = cv2.GaussianBlur(diff_image,(5,5),0)
    ret, thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel = np.ones((4,4), np.uint8)
    kernel2 = np.ones((2,2),np.uint8)
    erosion = cv2.erode(thresh,kernel2,iterations = 1)
    closing = cv2.morphologyEx(erosion, cv2.MORPH_CLOSE, kernel,iterations=3)

    contours, hierarchy = cv2.findContours(closing.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)

    # calculate moments and extract size of contoour disregard small
    moment_dict = {}
    for index, cnt in enumerate(contours):
        moment_dict[index] = cv2.moments(cnt)

    obj_properties = {}
    for index, obj_moments in moment_dict.items():
        if obj_moments['m00'] >2000 and obj_moments['m00'] < 50000:
            area = obj_moments['m00']
            cx = obj_moments['m10'] / obj_moments['m00']
            cy = obj_moments['m01'] / obj_moments['m00']
            peri = cv2.arcLength(contours[index], True)
            q = (peri/(math.sqrt(area)))
            props = {}
            props['q'] = q
            props['peri'] = peri
            props['area'] = area
            props['cx'] = cx
            props['cy'] = cy
            obj_properties[index] = props

        all_the_q = [v['q'] for k, v in obj_properties.items()]
        min_q = min(all_the_q, default=0)
        max_q = max(all_the_q, default=0)
        range_q = max_q - min_q

    cmap = plt.cm.get_cmap('terrain')
    for index, prop in obj_properties.items():
        if range_q > 0:
            v = (prop['q'] - min_q) / range_q
            r, g, b, a = [int(x) for x in cmap(v, bytes=True)]

            x,y,w,h = cv2.boundingRect(contours[index])  
            cv2.rectangle(orginal_img[i],(x,y),(x+w,y+h),(255,255,255),3)

            ctimg= orginal_img[i][y:y+h, x:x+w]

    cv2.imshow('video', orginal_img[i])
    if cv2.waitKey(33) == 27:
        break

    cv2.destroyAllWindows()

4.5.1


In [ ]:
dilater = cv2.dilate(opening, kernel2, iterations=7)
 